# 🐳Happywhale 🐬
Identify and group all images that contain the same individual through time.

# Initial Setup

## Importing libraries

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import sklearn

# import cudf, cupy, cuml
# from cuml.neighbors import NearestNeightbors
# from cuml.manifold import TSNE, PCE

import imageio
import gc
import random
from tqdm.notebook import tqdm

tqdm.pandas()

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
from matplotlib import animation, rc
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
import PIL
import plotly
import cv2
from PIL import Image, ImageEnhance
import plotly.io as pio
print(pio.renderers)

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow addons version: {tfa.__version__}")
print(f"SkLearn version: {sklearn.__version__}")

In [4]:
print("Notebbook Color Scheme:")
color_palette = sns.color_palette("viridis", as_cmap=True)

sns.color_palette("viridis")

In [ ]:
class color:
   GREEN = '\033[92m'
   BOLD = '\033[1m'
   END = '\033[0m'
    
class Config:
    import os
    ROOT_LOGDIR = os.path.join(os.curdir, "logdir") # Logs dir

    def __init__(self, project=None, model=None,
                 disable_wandb=False, seed=42):
        import os
        import time
        import wandb
        import json
        import random
        
        self.DISABLE_WANDB = disable_wandb
        self.BATCH_SIZE = 32
        self.FOLD = 4
        self.EPOCHS = 30
        self.MODEL_PATH = f"{model}.h5"     
        
        """ Attempt to be Reproducible """
        os.environ['PYTHONHASHSEED'] = str(seed)
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        
        """Get the current run logdir inside root_logdir and run_id
           Weights and Bias experiment tracking"""
        self.run_id = model + time.strftime("-run_%Y_%m_%d-%H_%M_%S") if model else time.strftime("run_%Y_%m_%d-%H_%M_%S")            
        if not disable_wandb: 
            self.WANDB_RUN = wandb.init(project=project, name=self.run_id, config=vars(self),
                                        sync_tensorboard=True)
            self.TENSORBOARD_LOGDIR = os.path.join(Config.ROOT_LOGDIR, self.run_id)
            
        print(color.BOLD + 'Config created with run ID: ' + color.END + color.BOLD + color.GREEN + self.run_id + color.END)
        
    def log_artifact(self, artifact_name, type_, file_path):
        """Log a artifact like preprocess file to wandb"""
        if self.DISABLE_WANDB: return
        artifact = wandb.Artifact(artifact_name, type=type_)
        artifact.add_file(file_path)
        self.WANDB_RUN.log_artifact(artifact)
    
    def finish(self):
        """Call this function to finish this run/experiment"""
        self.WANDB_RUN.finish()
        
        
config = Config(project='Happywhale', model='')

# Data Preparation
## Metadata Cleaning
**Species column typos adjustment**
- `bottlenose_dolpin` -> `bottlenose_dolphin`
- `kiler_whale` -> `killer_whale`
- `beluga` -> `beluga_whale`
- `globis` & `pilot_whale` -> `short_finned_pilot_whale` (due to extreme similarities [according to this discussion](https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/305909)

In [ ]:
# Importing the training data
train_df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")

# Adjust typos in "species" column
train["species"] = train["species"].replace({
    "bottlenose_dolphin": "bottlenode_dolphin",
    "kiler_whale": "killer_whale",
    "beluga": "beluga_whale",
    "globis": "short_finned_pilot_whale",
    "pilot_whale": "short_finned_pilot_whale"
})

# Create a "class" columns
train["class"] = train["species"].applu(lambda x: x.split("_")[-1])

# Train path
TRAIN_PATH = "../input/happy-whale-and-dolphin/train_images/"
train["path"] = TRAIN_PATH + train["image"]

